In [1]:
import sys
import numpy as np 
import xarray as xr
from scipy.interpolate import griddata
import matplotlib.pylab as plt
import netCDF4 as nc

/Users/leguillou/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
myPath = '/Users/leguillou/WORK/Developpement/'
dir_data =  myPath + "DATA/eNATL60/merged_outputs/hourly/"
dir_output = myPath + "DATA/eNATL60/merged_outputs/hourly/"
filename_data = "eNATL60_GULFSTREAM_SSH.1h_y2009m07d01_y2010m06d30.nc"

In [3]:
xrin = xr.open_mfdataset(dir_data + filename_data,decode_times=True)
print(xrin)

<xarray.Dataset>
Dimensions:            (bnds: 2, time_counter: 8760, x: 613, y: 770)
Coordinates:
    nav_lon            (y, x) float32 dask.array<shape=(770, 613), chunksize=(770, 613)>
    nav_lat            (y, x) float32 dask.array<shape=(770, 613), chunksize=(770, 613)>
  * time_counter       (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2010-06-30T23:30:00
Dimensions without coordinates: bnds, x, y
Data variables:
    time_counter_bnds  (time_counter, bnds) datetime64[ns] dask.array<shape=(8760, 2), chunksize=(8760, 2)>
    sossheig           (time_counter, y, x) float32 dask.array<shape=(8760, 770, 613), chunksize=(8760, 770, 613)>
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    history:      Fri Nov 08 11:28:28 2019: cdo cat eNATL60GULFSTREAM-BLBT02_...
    Conventions:  CF-1.6
    name:         /scratch/tmp/3746956/eNATL60-BLBT02_1h_20090630_20090704_gr...
    description:  ocean T grid variables
    title:        ocean T

In [4]:
SSH = xrin['sossheig'][:]
timestamps = xrin['time_counter'][:]
navlat = xrin['nav_lat']
navlon = xrin['nav_lon']

## Define regular grid

In [5]:
lon_min,lon_max = -65,-55
lat_min,lat_max = 33,43
dlat, dlon = 1/30, 1/30

In [6]:
lon_reg = np.arange(lon_min,lon_max,dlon)
lat_reg = np.arange(lat_min,lat_max,dlat)

In [7]:
lat_2d_reg, lon_2d_reg = np.meshgrid(lat_reg,lon_reg)
NY,NX = lat_2d_reg.shape

## Interpolation

In [8]:
NT = timestamps.size
ncout = nc.Dataset(dir_output + filename_data[:-3] + '_degraded30.nc', 'w', format='NETCDF3_CLASSIC')
ncout.createDimension('time', NT//24)
ncout.createDimension('x', NX)
ncout.createDimension('y', NY) 
nctime = ncout.createVariable('time', 'f', ('time',))
nctime.units = 'seconds since 1900-1-1 00:00:00'
nclon = ncout.createVariable('lon', 'f', ('y','x',))
nclat = ncout.createVariable('lat', 'f', ('y','x',))  
ncssh = ncout.createVariable('ssh_degraded', 'f', ('time','y','x',))  

nclon[:] = lon_2d_reg
nclat[:] = lat_2d_reg
for t in range(NT):
    ts = (timestamps[t] - np.datetime64('1900-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    nctime[t] = ts
    ncssh[t,:,:] = griddata((navlat.values.ravel(),navlon.values.ravel()), SSH[t].values.ravel(), (lat_2d_reg.ravel(),lon_2d_reg.ravel()), method='cubic').reshape((NY,NX))
ncout.close()

/Users/leguillou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  from ipykernel import kernelapp as app
/Users/leguillou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  from ipykernel import kernelapp as app
/Users/leguillou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  from ipykernel import kernelapp as app
/Users/leguillou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  from ipykernel import kernelapp as app
/Users/leguillou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: Dep

KeyboardInterrupt: 

In [9]:
t

6